In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

from utils.features import signature
import datasets

In [2]:
dataset_sagittal = datasets.Tumors(path="./data/tdsc/tumors_3d")

# Signature of middle slice

In [3]:
signature_dataset = []
for x,m,y in dataset_sagittal:
    w,h,d = x.shape
    sig = signature(m[:,:,d//2] ,res=1)
    signature_dataset.append((sig, y))

In [4]:
x,y = signature_dataset[0]
for i in range(len(signature_dataset)):
    x,y = signature_dataset[i]
    signature_dataset[i] = (x.tolist(), y)

In [26]:
X,Y = zip(*signature_dataset)

In [43]:
acc = 0 
cfm = np.zeros([2,2], dtype=np.uint8)

for i in range(len(X)):

    x_test = X[i]
    y_test = Y[i]
    x_train = X[:i]+X[i+1:]
    y_train = Y[:i]+Y[i+1:]
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    prediction = clf.predict(x_train)
    accuracy = accuracy_score(y_train, prediction)
    print(accuracy)
    predict = clf.predict([x_test])[0]
    if predict == y_test:
        acc += 1
    cfm[y_test,predict] += 1

print(f"test accuracy is: {acc/100}")
print(cfm)

0.6868686868686869
0.6868686868686869
0.7070707070707071
0.6868686868686869
0.696969696969697
0.6868686868686869
0.6868686868686869
0.696969696969697
0.6868686868686869
0.6868686868686869
0.7171717171717171
0.7070707070707071
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.696969696969697
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.7070707070707071
0.6868686868686869
0.6868686868686869
0.696969696969697
0.7070707070707071
0.696969696969697
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.696969696969697
0.7070707070707071
0.6868686868686869
0.6868686868686869
0.7070707070707071
0.696969696969697
0.696969696969697
0.7070707070707071
0.7070707070707071
0.6868686868686869
0.696969696969697
0.7070707070707071
0.696969696969697
0.7070707070707071
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.696969696969697
0.696969696969697
0.696

# Normalized middle slice signature


$$ X_{std} = (X - X_{min}) / (X_{max} - X_{min}) $$
$$ X_{scaled} = X_{std} * (max - min) + min $$

In [36]:
X = list(X)
Y = list(Y)
scaler = MinMaxScaler()
normalized_X = scaler.fit_transform(X).tolist()

In [41]:
acc = 0 
cfm = np.zeros([2,2], dtype=np.uint8)
for i in range(len(X)):

    x_test = normalized_X[i]
    y_test = Y[i]
    x_train = normalized_X[:i]+normalized_X[i+1:]
    y_train = Y[:i]+Y[i+1:]
    clf = svm.SVC()
    clf.fit(x_train,y_train)
    prediction = clf.predict(x_train)
    accuracy = accuracy_score(y_train, prediction)
    print(accuracy)
    predict = clf.predict([x_test])[0]
    if predict == y_test:
        acc += 1
    cfm[y_test,predict] += 1

print(f"test accuracy is: {acc/100}")
print(cfm)

0.6868686868686869
0.6868686868686869
0.7070707070707071
0.696969696969697
0.6868686868686869
0.6868686868686869
0.696969696969697
0.6868686868686869
0.696969696969697
0.7171717171717171
0.7070707070707071
0.6868686868686869
0.696969696969697
0.696969696969697
0.6868686868686869
0.696969696969697
0.7070707070707071
0.6767676767676768
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6767676767676768
0.696969696969697
0.6868686868686869
0.7070707070707071
0.696969696969697
0.6868686868686869
0.696969696969697
0.696969696969697
0.7070707070707071
0.696969696969697
0.6868686868686869
0.696969696969697
0.696969696969697
0.7070707070707071
0.696969696969697
0.696969696969697
0.7070707070707071
0.6868686868686869
0.696969696969697
0.696969696969697
0.7070707070707071
0.6868686868686869
0.6868686868686869
0.7070707070707071
0.696969696969697
0.6868686868686869
0.6868686868686869
0.696969696969697
0.696969696969697
0.6868686868686869
0.7070707070707071
0.696969696969697
0.696969696969

## Use multiple slices signatures

In [4]:
signature_dataset = []
for x,m,y in dataset_sagittal:
    w,h,d = x.shape
    sig = np.zeros([3*360], dtype=np.float32)
    j = 0
    for i in range(d//2-1, d//2+1):
        sig[360*j:360*(j+1)] = signature(m[:,:,d//2] ,res=1)
        j += 1
    signature_dataset.append((sig.tolist(), y))

In [5]:
X, Y = zip(*signature_dataset)
X = list(X)
Y = list(Y)

In [6]:
acc = 0 
cfm = np.zeros([2,2], dtype=np.uint8)

for i in range(len(X)):

    x_test = X[i]
    y_test = Y[i]
    x_train = X[:i]+X[i+1:]
    y_train = Y[:i]+Y[i+1:]
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    prediction = clf.predict(x_train)
    accuracy = accuracy_score(y_train, prediction)
    print(accuracy)
    predict = clf.predict([x_test])[0]
    if predict == y_test:
        acc += 1
    cfm[y_test,predict] += 1

print(f"test accuracy is: {acc/100}")
print(cfm)

0.6767676767676768
0.6868686868686869
0.6868686868686869
0.696969696969697
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6868686868686869
0.6868686868686869
0.6767676767676768
0.6666666666666666
0.6868686868686869
0.6767676767676768
0.6868686868686869
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6666666666666666
0.6767676767676768
0.6767676767676768
0.6868686868686869
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.696969696969697
0.6868686868686869
0.6868686868686869
0.696969696969697
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6868686868686869
0.6767676767676768
0.6868686868686869
0.6868686868686869
0.6767676767676768
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6767676767676768
0.6767676767676768
0.6868686868686869
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6868686868686869
0.6868686868686869
0.6767676767676768
0.6666666666666666
0.6868686868686

### Normalized multiple slices

In [7]:
X = list(X)
Y = list(Y)
scaler = MinMaxScaler()
normalized_X = scaler.fit_transform(X).tolist()

In [8]:
acc = 0 
cfm = np.zeros([2,2], dtype=np.uint8)
for i in range(len(X)):

    x_test = normalized_X[i]
    y_test = Y[i]
    x_train = normalized_X[:i]+normalized_X[i+1:]
    y_train = Y[:i]+Y[i+1:]
    clf = svm.SVC()
    clf.fit(x_train,y_train)
    prediction = clf.predict(x_train)
    accuracy = accuracy_score(y_train, prediction)
    print(accuracy)
    predict = clf.predict([x_test])[0]
    if predict == y_test:
        acc += 1
    cfm[y_test,predict] += 1

print(f"test accuracy is: {acc/100}")
print(cfm)

0.6767676767676768
0.6767676767676768
0.6868686868686869
0.6767676767676768
0.6666666666666666
0.6767676767676768
0.6868686868686869
0.6565656565656566
0.6767676767676768
0.6868686868686869
0.6767676767676768
0.6666666666666666
0.6767676767676768
0.6868686868686869
0.6868686868686869
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6868686868686869
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6767676767676768
0.6868686868686869
0.696969696969697
0.6767676767676768
0.6868686868686869
0.6868686868686869
0.6868686868686869
0.6767676767676768
0.6666666666666666
0.6767676767676768
0.6868686868686869
0.6767676767676768
0.6767676767676768
0.6666666666666666
0.6666666666666666
0.6868686868686869
0.6767676767676768
0.6767676767676768
0.6666666666666666
0.6767676767676768
0.6868686868686869
0.6666666666666666
0.6868686868686869
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.6767676767676768
0.69696969696